In [ ]:

# Python Libraries
%matplotlib inline
import pickle
import numpy as np
import pandas as pd
import matplotlib
from keras.datasets import cifar10
from keras import backend as K
import os,sys
#import Pillow

# Custom Networks
#from networks.lenet import LeNet
#sys.path.append('./')
from networks.pure_cnn import PureCnn
from networks.network_in_network import NetworkInNetwork
from networks.resnet import ResNet
from networks.densenet import DenseNet
from networks.wide_resnet import WideResNet
from networks.capsnet import CapsNet

# Helper functions
from differential_evolution import differential_evolution
import helper
import scipy.misc

#from scipy.misc import imsave

matplotlib.style.use('ggplot')
np.random.seed(100)

def load_results():
    with open('networks/results/targeted_results.pkl', 'rb') as file:
        targeted = pickle.load(file)
    return targeted

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

K.tensorflow_backend._get_available_gpus()
#nin = NetworkInNetwork()
resnet = ResNet()
#densenet = DenseNet()

models = [resnet]


In [ ]:
network_stats, correct_imgs = helper.evaluate_models(models, x_test, y_test)
correct_imgs = pd.DataFrame(correct_imgs, columns=['name', 'img', 'label', 'confidence', 'pred'])
network_stats = pd.DataFrame(network_stats, columns=['name', 'accuracy', 'param_count'])

network_stats

In [ ]:
targeted = load_results()

In [ ]:
columns = ['model', 'pixels', 'image', 'true', 'predicted', 'success', 'cdiff', 'prior_probs', 'predicted_probs', 'perturbation']

targeted_results = pd.DataFrame(targeted, columns=columns)

In [ ]:

stats = []
for model in models:
    print(models)
    val_accuracy = np.array(network_stats[network_stats.name == model.name].accuracy)[0]
    m_result = targeted_results[targeted_results.model == model.name]
    pixels = list(set(m_result.pixels))
    print(model,pixels)
    for pixel in pixels:
        p_result = m_result[m_result.pixels == pixel]
        success_rate = len(p_result[p_result.success]) / len(p_result)
        print(len(p_result[p_result.success]))
        print(len(p_result))
        stats.append([model.name, val_accuracy, pixel, success_rate])

#helper.attack_stats(targeted_results, models, network_stats)

In [ ]:
model.name

In [ ]:
def visualize_attack(df, class_names):
    _, (x_test, _) = cifar10.load_data()

    results = df[df.success]#.sample(9)
    print(results.shape)
    
    z = zip(results.perturbation, x_test[results.image])
    images = np.array([perturb_image(p, img)[0]
                       for p,img in z])
     
    labels_true = np.array(results.true)
    labels_pred = np.array(results.predicted)
    #titles = np.array(results.model)

In [ ]:
def perturb_image(xs, img):
    # If this function is passed just one perturbation vector,
    # pack it in a list to keep the computation the same
    if xs.ndim < 2:
        xs = np.array([xs])
    
    # Copy the image n == len(xs) times so that we can 
    # create n new perturbed images
    tile = [len(xs)] + [1]*(xs.ndim+1)
    imgs = np.tile(img, tile)
    
    # Make sure to floor the members of xs as int types
    xs = xs.astype(int)
    
    for x,img in zip(xs, imgs):
        # Split x into an array of 5-tuples (perturbation pixels)
        # i.e., [[x,y,r,g,b], ...]
        pixels = np.split(x, len(x) // 5)
        for pixel in pixels:
            # At each pixel's x,y position, assign its rgb value
            x_pos, y_pos, *rgb = pixel
            img[x_pos, y_pos] = rgb
    
    return imgs

In [ ]:
# print('Targeted Attack')
# helper.visualize_attack(targeted_results, class_names)
model_id = model.name
pixel_id = 5
_, (x_test, _) = cifar10.load_data()
results = targeted_results[targeted_results.success]
#print(results.model)
results = results[results.model==model.name]
results = results[results.pixels==pixel_id]
print(results.shape)

z = zip(results.perturbation, x_test[results.image])
images = np.array([perturb_image(p, img)[0] for p,img in z])
labels_true = np.array(results.true)
labels_pred = np.array(results.predicted)

image_id = [id for id in results.image]
count = 0
for i in range(len(labels_true)):
    name = str(count)+'_'+str(image_id[i])+'_'+str(labels_true[i])+'_'+str(labels_pred[i])+'.png'
    image = images[i]
    print(name)
    #import cv2 as cv
    out_path = 'non_'+model_id+'_p'+str(pixel_id)+'/'
    if not os.path.exists(out_path):
        os.system("mkdir -p %s"%(out_path))
    out_name = out_path + name
    #cv.imwrite(out_name, image)
    from PIL import Image
    im = Image.fromarray(image)
    print(out_name)
    im.save(out_name)
    count += 1
    #import imageio
    #imageio.imwrite(out_name,image)
    #imsave(out_name,image)

In [ ]:
results